### RFM 분석 실습
- Recency: 얼마나 최근에 구매했는가
- Frequency: 얼마나 자주 구매했는가
- Monetary: 얼마나 많은 금액을 지출했는가

### 고객 분석
- InvoiceNo: 거래 번호
- StockCode: 상품 코드
- Description: 상품명 
- InvoiceDate: 거래 날짜
- UnitPrice: 상품 단가
- CustomerID: 고객 번호
- Country: 국가명

In [1]:
import pandas as pd

ci_df = pd.read_csv('./datasets/customer_invoice_data.csv')
ci_df

FileNotFoundError: [Errno 2] No such file or directory: './datasets/customer_invoice_data.csv'

In [ ]:
ci_df.isna().sum()

In [ ]:
ci_df = ci_df.drop(index=ci_df[ci_df['CustomerID'].isna()].index, axis=1)

In [ ]:
import numpy as np

ci_df = ci_df.astype({'CustomerID': np.int16})
ci_df

In [ ]:
ci_df.isna().sum()

In [ ]:
ci_df = ci_df.reset_index(drop=True)
ci_df

In [ ]:
ci_df.duplicated().sum()

In [ ]:
ci_rf_df = ci_df[ci_df['Quantity'] >= 0]
ci_rf_df

In [ ]:
ci_rf_df.loc[:, 'Recency'] = pd.to_datetime(ci_df['InvoiceDate'])\
                    .apply(lambda x: (pd.to_datetime('2011-12-10') - x).days)
ci_rf_df

In [ ]:
ci_rf_df = ci_rf_df.drop(['InvoiceDate'], axis=1)

In [ ]:
rf_df = ci_rf_df.groupby('CustomerID').agg(Recency=('Recency', 'min'), Frequency=('CustomerID', 'count'))
rf_df

In [ ]:
m_df = ci_df.copy()
m_df.loc[:, 'Monetary'] = m_df['Quantity'] * m_df['UnitPrice']
m_df

In [ ]:
m_df = m_df.groupby('CustomerID')[['Monetary']].sum()
m_df

In [ ]:
rf_df = rf_df.reset_index()
rf_df

In [ ]:
m_df = m_df.reset_index()
m_df

In [ ]:
rfm_df = rf_df.merge(m_df, on='CustomerID')
rfm_df

In [ ]:
rfm_df.iloc[rfm_df[rfm_df['Monetary'] < 0].index] = 0
rfm_df[rfm_df['Monetary'] < 0]

In [ ]:
rfm_df

In [ ]:
from sklearn.preprocessing import MinMaxScaler

rfm_scale = MinMaxScaler()
rfm_scale = rfm_scale.fit_transform(rfm_df[['Recency', 'Frequency', 'Monetary']])

In [ ]:
rfm_df[['Recency', 'Frequency', 'Monetary']] = pd.DataFrame(rfm_scale, columns=['Recency', 'Frequency', 'Monetary'])
rfm_df

In [ ]:
rfm_df['Recency'] = 1 - rfm_df['Recency']

In [ ]:
rfm_df

In [ ]:
rfm_df.loc[:, 'TotalScore'] = rfm_df['Recency'] + rfm_df['Frequency'] + rfm_df['Monetary']
rfm_df

In [ ]:
ci_rfm_df = ci_df.merge(rfm_df[['CustomerID', 'TotalScore']], on='CustomerID')
ci_rfm_df

In [ ]:
import numpy as np

l1, l2, l3, l4 = np.percentile(ci_rfm_df['TotalScore'], [20, 40, 70, 90])
print(l1, l2, l3, l4)

In [ ]:
def get_level(x):
    if x <= l1:
        return 5
    if x <= l2:
        return 4
    if x <= l3:
        return 3
    if x <= l4:
        return 2
    return 1

In [ ]:
ci_rfm_df['Level'] = ci_rfm_df['TotalScore'].apply(get_level)
ci_rfm_df

In [ ]:
ci_rfm_df['Level'] = ci_rfm_df['Level'].replace([5, 4, 3, 2, 1], 
                                                ['Bronze', 'Silver', 'Gold', 'Diamond', 'VIP'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

order = ['Bronze', 'Silver', 'Gold', 'Diamond', 'VIP']

sns.countplot(x = 'Level', data=ci_rfm_df, palette='muted', order=order)
plt.title('Level Count')
plt.show()